In [ ]:
import pandas as pd
import numpy as np

Создаем массив дат, для которых имеются наблюдения

In [ ]:
dates = []
for year in range(2008,2018):
    for month in range(1,13):
        if month>=10:
            dates.append(str(year)+'-'+str(month)+'-01')
        else:
            dates.append(str(year)+'-0'+str(month)+'-01')
dates = dates[2:-9]

Открываем файл с таблицей макроэкономических показателей

In [ ]:
macro = pd.read_csv('macro.csv')

Открываем таблицы с показателями за различные месяцы и добавляем к ним макроэкономические показатели за этот месяц. При этом создаем таблицы для каждого банка и последовательно, по датам, заносим туда информацию для банка за каждый месяц

In [ ]:
banks = {}
for j,now in enumerate(dates):
    if j%10==0:
        print(j)
    base = pd.read_csv('banki.ru/'+now+'.csv',index_col=0)
    base = base.transpose()
    for column in macro.columns:
        base[column] = macro[column][108-j]
    for i in range(len(base)):
        if base.index[i][:3]=='Unn':
            continue
        num = int(base.index[i])
        if num in banks.keys():
            banks[num] = banks[num].append(base.take([i]))
        else:
            banks.update({num:base.take([i])})

Убираем банки с потерянными наблюдениями

In [ ]:
for name in banks.keys():
    banks[name]['por'] = (banks[name]['year']-2008)*12+banks[name]['month']
    if len(banks[name])-(banks[name]['por'][len(banks[name])-1]-banks[name]['por'][0]+1)!=0:
        print(name,len(banks[name])-(banks[name]['por'][len(banks[name])-1]-banks[name]['por'][0]+1))
        del banks[name]

Считаем пропуски для признаков

In [ ]:
non = {}
for i in banks[1].columns:
    non[i] = 0
for j,name in enumerate(banks.keys()):
    if j%50==0:
        print(j)
    for i in banks[1].columns:
        non[i] += np.sum(banks[name][i].isnull())

Определяем "хорошие" признаки, как признаки, где менее 900 пропусков

In [ ]:
good_feat = []
for i,j in non.items():
    if j<900:
        good_feat.append(i)

Оставляем только "хорошие" признаки

In [ ]:
for name in banks.keys():
    banks[name] = banks[name][good_feat]
    del banks[name]['por']

Сохраняем результат файлы с названиями по номерам лицензий

In [ ]:
for name in banks.keys():
    banks[name].to_csv('data/'+str(name)+'.csv',index=False)

Эти признаки были "хорошими", но для них имелся пропуск в один из месяцев для всех банков

In [ ]:
bad = ['180','400','410','500','510','600']

Исправляем этот пропуск

In [ ]:
for name in banks.keys():
    if banks[name].index[0]<79 and banks[name].index[len(banks[name])-1]>79:
        for j in bad:
            banks[name][j][79] = (banks[name][j][78]+banks[name][j][80])/2
    elif banks[name].index[0]==79:
        for j in bad:
            banks[name][j][79] = banks[name][j][80]
    elif banks[name].index[len(banks[name])-1]==79:
        for j in bad:
            banks[name][j][79] = banks[name][j][78]